In [ ]:
print("hello")

In [ ]:
import os
import gradio as gr
import torch
import numpy as np
import imageio
from PIL import Image
import uuid

from draggan import utils
from draggan.draggan import drag_gan
from draggan import draggan as draggan

device = "cuda"


SIZE_TO_CLICK_SIZE = {1024: 8, 512: 5, 256: 2}

CKPT_SIZE = {
    "stylegan2/stylegan2-ffhq-config-f.pkl": 1024,
    "stylegan2/stylegan2-cat-config-f.pkl": 256,
    "stylegan2/stylegan2-church-config-f.pkl": 256,
    "stylegan2/stylegan2-horse-config-f.pkl": 256,
    "ada/ffhq.pkl": 1024,
    "ada/afhqcat.pkl": 512,
    "ada/afhqdog.pkl": 512,
    "ada/afhqwild.pkl": 512,
    "ada/brecahad.pkl": 512,
    "ada/metfaces.pkl": 512,
    "human/stylegan_human_v2_512.pkl": 512,
    "human/stylegan_human_v2_1024.pkl": 1024,
    "self_distill/bicycles_256_pytorch.pkl": 256,
    "self_distill/dogs_1024_pytorch.pkl": 1024,
    "self_distill/elephants_512_pytorch.pkl": 512,
    "self_distill/giraffes_512_pytorch.pkl": 512,
    "self_distill/horses_256_pytorch.pkl": 256,
    "self_distill/lions_512_pytorch.pkl": 512,
    "self_distill/parrots_512_pytorch.pkl": 512,
}

DEFAULT_CKPT = "ada/afhqcat.pkl"

import logging

# Configure logging
logging.basicConfig(filename='./debug.log', level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger=logging.getLogger(__name__)


def to_image(tensor):
    tensor = tensor.squeeze(0).permute(1, 2, 0)
    arr = tensor.detach().cpu().numpy()
    arr = (arr - arr.min()) / (arr.max() - arr.min())
    arr = arr * 255
    return arr.astype("uint8")


def add_points_to_image(image, points, size=5):
    image = utils.draw_handle_target_points(image, points["handle"], points["target"], size)
    return image


def on_click(image, target_point, points, size, evt: gr.SelectData):
    if target_point:
        points["target"].append([evt.index[1], evt.index[0]])
        image = add_points_to_image(image, points, size=SIZE_TO_CLICK_SIZE[size])
        return image, not target_point
    points["handle"].append([evt.index[1], evt.index[0]])
    image = add_points_to_image(image, points, size=SIZE_TO_CLICK_SIZE[size])
    return image, not target_point


def on_drag(model, points, max_iters, state, size, mask, lr_box):
    # logger.info("model: %s", model)
    # logger.info("points: %s", points)
    # logger.info("max_iters: %s", max_iters)
    # logger.info("state: %s", state)
    # logger.info("size: %s", size)
    # logger.info("mask: %s", mask)
    # logger.info("lr_box: %s", lr_box)

    if len(points["handle"]) == 0:
        raise gr.Error("You must select at least one handle point and target point.")
    if len(points["handle"]) != len(points["target"]):
        raise gr.Error(
            "You have uncompleted handle points, try to selct a target point or undo the handle point."
        )
    max_iters = int(max_iters)
    W = state["W"]

    handle_points = [torch.tensor(p, device=device).float() for p in points["handle"]]
    target_points = [torch.tensor(p, device=device).float() for p in points["target"]]

    if mask.get("mask", None) is not None:
        mask = Image.fromarray(mask["mask"]).convert("L")
        mask = np.array(mask) == 255

        mask = torch.from_numpy(mask).float().to(device)
        mask = mask.unsqueeze(0).unsqueeze(0)
    else:
        mask = None

    step = 0
    output_directory = "./out/"
    os.makedirs(output_directory, exist_ok=True)

    for image, W, handle_points in drag_gan(
        W, model["G"], handle_points, target_points, mask, max_iters=max_iters, lr=lr_box
    ):
        points["handle"] = [p.cpu().numpy().astype("int") for p in handle_points]
        image = add_points_to_image(image, points, size=SIZE_TO_CLICK_SIZE[size])

        state["history"].append(image)
        step += 1

        output_path = os.path.join(output_directory, f"output_{step}.png")
        Image.fromarray(image).save(output_path)

        yield image, state, step


def on_reset(points, image, state):
    return {"target": [], "handle": []}, state["img"], False


def on_undo(points, image, state, size):
    image = state["img"]

    if len(points["target"]) < len(points["handle"]):
        points["handle"] = points["handle"][:-1]
    else:
        points["handle"] = points["handle"][:-1]
        points["target"] = points["target"][:-1]

    image = add_points_to_image(image, points, size=SIZE_TO_CLICK_SIZE[size])
    return points, image, False


def on_change_model(selected, model):
    size = CKPT_SIZE[selected]

    G = draggan.load_model(utils.get_path(selected), device=device)
    model = {"G": G}
    W = draggan.generate_W(
        G,
        seed=int(1),
        device=device,
        truncation_psi=0.8,
        truncation_cutoff=8,
    )
    img, _ = draggan.generate_image(W, G, device=device)

    state = {"W": W, "img": img, "history": []}

    return model, state, img, img, size


def on_new_image(model, seed):
    G = model["G"]
    W = draggan.generate_W(
        G,
        seed=int(seed),
        device=device,
        truncation_psi=0.8,
        truncation_cutoff=8,
    )
    img, _ = draggan.generate_image(W, G, device=device)

    state = {"W": W, "img": img, "history": []}

    points = {"target": [], "handle": []}
    target_point = False
    return img, img, state, points, target_point


def on_max_iter_change(max_iters):
    return gr.update(maximum=max_iters)


def on_save_files(image, state):
    os.makedirs("draggan_tmp", exist_ok=True)
    image_name = f"draggan_tmp/image_{uuid.uuid4()}.png"
    video_name = f"draggan_tmp/video_{uuid.uuid4()}.mp4"
    imageio.imsave(image_name, image)
    imageio.mimsave(video_name, state["history"])
    return [image_name, video_name]


def on_show_save():
    return gr.update(visible=True)


def on_image_change(model, image_size, image):
    image = Image.fromarray(image)
    result = inverse_image(model.g_ema, image, image_size=image_size)
    result["history"] = []
    image = to_image(result["sample"])
    points = {"target": [], "handle": []}
    target_point = False
    return image, image, result, points, target_point


def on_mask_change(mask):
    return mask["image"]


def on_select_mask_tab(state):
    img = to_image(state["sample"])
    return img


def main():
    torch.cuda.manual_seed(25)

    with gr.Blocks() as demo:
        G = draggan.load_model(utils.get_path(DEFAULT_CKPT), device=device)
        model = gr.State({"G": G})
        W = draggan.generate_W(
            G,
            seed=int(1),
            device=device,
            truncation_psi=0.8,
            truncation_cutoff=8,
        )
        img, F0 = draggan.generate_image(W, G, device=device)

        state = gr.State({"W": W, "img": img, "history": []})
        points = gr.State({"target": [], "handle": []})
        size = gr.State(CKPT_SIZE[DEFAULT_CKPT])
        target_point = gr.State(False)

        with gr.Row():
            with gr.Column(scale=0.3):
                with gr.Accordion("Model"):
                    model_dropdown = gr.Dropdown(
                        choices=list(CKPT_SIZE.keys()), value=DEFAULT_CKPT, label="StyleGAN2 model"
                    )
                    seed = gr.Number(value=1, label="Seed", precision=0)
                    new_btn = gr.Button("New Image")
                with gr.Accordion("Drag"):
                    with gr.Row():
                        lr_box = gr.Number(value=2e-3, label="Learning Rate")
                        max_iters = gr.Slider(1, 500, 20, step=1, label="Max Iterations")

                    # with gr.Row():
                    #     with gr.Column(min_width=100):
                    #         reset_btn = gr.Button('Reset All')
                    #     with gr.Column(min_width=100):
                    #         undo_btn = gr.Button('Undo Last')
                    with gr.Row():
                        btn = gr.Button("Drag it", variant="primary")

                with gr.Accordion("Save", visible=False) as save_panel:
                    files = gr.Files(value=[])

                progress = gr.Slider(value=0, maximum=20, label="Progress", interactive=False)

            with gr.Column():
                with gr.Tabs():
                    with gr.Tab("Setup Handle Points", id="input"):
                        image = gr.Image(img).style(height=512, width=512)
                    with gr.Tab("Draw a Mask", id="mask") as masktab:
                        mask = gr.ImageMask(img, label="Mask").style(height=512, width=512)

        image.select(on_click, [image, target_point, points, size], [image, target_point])
        image.upload(on_image_change, [model, size, image], [image, mask, state, points, target_point])
        mask.upload(on_mask_change, [mask], [image])
        btn.click(
            on_drag,
            inputs=[model, points, max_iters, state, size, mask, lr_box],
            outputs=[image, state, progress],
        ).then(on_show_save, outputs=save_panel).then(on_save_files, inputs=[image, state], outputs=[files])
        # reset_btn.click(on_reset, inputs=[points, image, state], outputs=[points, image, target_point])
        # undo_btn.click(on_undo, inputs=[points, image, state, size], outputs=[points, image, target_point])
        model_dropdown.change(
            on_change_model, inputs=[model_dropdown, model], outputs=[model, state, image, mask, size]
        )
        new_btn.click(on_new_image, inputs=[model, seed], outputs=[image, mask, state, points, target_point])
        max_iters.change(on_max_iter_change, inputs=max_iters, outputs=progress)
        masktab.select(lambda: gr.update(value=None), outputs=[mask]).then(
            on_select_mask_tab, inputs=[state], outputs=[mask]
        )
    return demo



___

In [ ]:
# import shutil

In [ ]:
# OUTPUT_DIR = "./out/"

# device = "cuda"

# shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# G = draggan.load_model(utils.get_path(DEFAULT_CKPT), device=device)
# W = draggan.generate_W(
#     G,
#     seed=int(1),
#     device=device,
#     truncation_psi=0.8,
#     truncation_cutoff=8,
# )
# img, F0 = draggan.generate_image(W, G, device=device)

In [ ]:
# # Initialize the model, points, max_iters, state, size, mask, and lr_box
# model = {"G": G}
# points = {'target': [[309, 57]], 'handle': [[379, 247]]}
# max_iters = 50
# state = {"W": W, "history": []}
# size = 512
# mask = {}
# lr_box = 0.01

# # Perform the drag operation
# for image, state, step in on_drag(model, points, max_iters, state, size, mask, lr_box):
#     print(f"Step: {step}")
#     # display(image)  # Or any other function to show the image


In [8]:
# Prediction interface for Cog ⚙️
# https://github.com/replicate/cog/blob/main/docs/python.md

from typing import List
from cog import BasePredictor, Input, Path
import os
import shutil
import torch
import numpy as np
import imageio
from PIL import Image
import uuid

from draggan import utils
from draggan.draggan import drag_gan
from draggan import draggan as draggan

SIZE_TO_CLICK_SIZE = {1024: 8, 512: 5, 256: 2}
CKPT_SIZE = {
    "stylegan2/stylegan2-ffhq-config-f.pkl": 1024,
    "stylegan2/stylegan2-cat-config-f.pkl": 256,
    "stylegan2/stylegan2-church-config-f.pkl": 256,
    "stylegan2/stylegan2-horse-config-f.pkl": 256,
    "ada/ffhq.pkl": 1024,
    "ada/afhqcat.pkl": 512,
    "ada/afhqdog.pkl": 512,
    "ada/afhqwild.pkl": 512,
    "ada/brecahad.pkl": 512,
    "ada/metfaces.pkl": 512,
    "human/stylegan_human_v2_512.pkl": 512,
    "human/stylegan_human_v2_1024.pkl": 1024,
    "self_distill/bicycles_256_pytorch.pkl": 256,
    "self_distill/dogs_1024_pytorch.pkl": 1024,
    "self_distill/elephants_512_pytorch.pkl": 512,
    "self_distill/giraffes_512_pytorch.pkl": 512,
    "self_distill/horses_256_pytorch.pkl": 256,
    "self_distill/lions_512_pytorch.pkl": 512,
    "self_distill/parrots_512_pytorch.pkl": 512,
}
DEFAULT_CKPT = "ada/afhqcat.pkl"
OUTPUT_DIR = "./out/"


class Predictor(BasePredictor):
    def setup(self):
        """Load the model into memory to make running multiple predictions efficient"""

        self.device = "cuda"

        shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
        os.makedirs(OUTPUT_DIR, exist_ok=True)

        self.G = draggan.load_model(utils.get_path(DEFAULT_CKPT), device=self.device)
        self.W = draggan.generate_W(
            self.G,
            seed=int(1),
            device=self.device,
            truncation_psi=0.8,
            truncation_cutoff=8,
        )
        self.img, self.F0 = draggan.generate_image(self.W, self.G, device=self.device)
        self.model = {"G": self.G}

    def add_points_to_image(self, image, points, size=5):
        image = utils.draw_handle_target_points(image, points["handle"], points["target"], size)
        return image

    def on_drag(self, model, points, max_iters, state, size, mask, lr_box):
        if len(points["handle"]) == 0:
            raise Exception("You must select at least one handle point and target point.")
        if len(points["handle"]) != len(points["target"]):
            raise Exception(
                "You have uncompleted handle points, try to selct a target point or undo the handle point."
            )
        max_iters = int(max_iters)
        W = state["W"]

        handle_points = [torch.tensor(p, device=self.device).float() for p in points["handle"]]
        target_points = [torch.tensor(p, device=self.device).float() for p in points["target"]]

        if mask.get("mask", None) is not None:
            mask = Image.fromarray(mask["mask"]).convert("L")
            mask = np.array(mask) == 255

            mask = torch.from_numpy(mask).float().to(self.device)
            mask = mask.unsqueeze(0).unsqueeze(0)
        else:
            mask = None

        step = 0
        for image, W, handle_points in drag_gan(
            W, model["G"], handle_points, target_points, mask, max_iters=max_iters, lr=lr_box
        ):
            points["handle"] = [p.cpu().numpy().astype("int") for p in handle_points]
            image = self.add_points_to_image(image, points, size=SIZE_TO_CLICK_SIZE[size])

            state["history"].append(image)
            step += 1

            output_path = os.path.join(OUTPUT_DIR, f"output_{step}.png")
            Image.fromarray(image).save(output_path)

            yield image, state, step

    def predict(
        self,
        # stylegan2_model: str = Input(
        #     description="Select a StyleGAN2 model", choices=list(CKPT_SIZE.keys()), default=DEFAULT_CKPT
        # ),
        # seed: int = Input(description="Specify the seed for image generation", ge=0, le=100, default=1),
        lr_box: float = Input(
            description="Specify the learning rate for the drag operation", ge=1e-4, le=1, default=2e-3
        ),
        max_iters: int = Input(
            description="Specify the maximum iterations for the drag operation", ge=1, le=500, default=20
        ),
    ) -> List[Path]:
        """Run a single prediction on the model"""

        shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
        os.makedirs(OUTPUT_DIR, exist_ok=True)

        points = {"target": [[309, 57]], "handle": [[379, 247]]}  # TODO: add as param
        state = {"W": self.W, "history": []}
        size = 512  # TODO: add as param
        mask = {}
        max_iters = 50
        lr_box = 2e-3

        _ = list(self.on_drag(self.model, points, max_iters, state, size, mask, lr_box))
        
        output_paths = [str((Path(OUTPUT_DIR) / file).absolute()) for file in os.listdir(OUTPUT_DIR) if file.endswith(".png")]

        if len(output_paths) == 0:
            raise Exception("No output images found. Please check the process.")

        return output_paths


p = Predictor()
p.setup();
out = p.predict()

Loading networks from "/home/sakib/draggan/checkpoints-pkl/ada/afhqcat.pkl"...
Registering hook for: b256
Registering hook for: b256
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Registering hook for: b256
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!


/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1296, in load
    keep_intermediates=keep_intermediates)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1534, in _jit_compile
    return _import_module_from_library(name, build_directory, is_python

Registering hook for: b256
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Loss: 0.03	Time: 55ms
Registering hook for: b256
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "u

/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1296, in load
    keep_intermediates=keep_intermediates)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1534, in _jit_compile
    return _import_module_from_library(name, build_directory, is_python

Failed!
Loss: 0.03	Time: 56ms
Registering hook for: b256
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Registering hook for: b256
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch p

...

/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1296, in load
    keep_intermediates=keep_intermediates)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1534, in _jit_compile
    return _import_module_from_library(name, build_directory, is_python

Registering hook for: b256
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Loss: 0.03	Time: 57ms
Registering hook for: b256
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "u

/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1296, in load
    keep_intermediates=keep_intermediates)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1534, in _jit_compile
    return _import_module_from_library(name, build_directory, is_python

Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Loss: 0.03	Time: 56ms
Registering hook for: b256
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"...

/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/home/sakib/DragGAN/draggan/stylegan2/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1296, in load
    keep_intermediates=keep_intermediates)
  File "/home/sakib/miniconda3/envs/draggan/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1534, in _jit_compile
    return _import_module_from_library(name, build_directory, is_python

In [9]:
out

['/home/sakib/DragGAN/out/output_19.png',
 '/home/sakib/DragGAN/out/output_15.png',
 '/home/sakib/DragGAN/out/output_9.png',
 '/home/sakib/DragGAN/out/output_38.png',
 '/home/sakib/DragGAN/out/output_33.png',
 '/home/sakib/DragGAN/out/output_50.png',
 '/home/sakib/DragGAN/out/output_5.png',
 '/home/sakib/DragGAN/out/output_18.png',
 '/home/sakib/DragGAN/out/output_7.png',
 '/home/sakib/DragGAN/out/output_17.png',
 '/home/sakib/DragGAN/out/output_14.png',
 '/home/sakib/DragGAN/out/output_32.png',
 '/home/sakib/DragGAN/out/output_42.png',
 '/home/sakib/DragGAN/out/output_8.png',
 '/home/sakib/DragGAN/out/output_31.png',
 '/home/sakib/DragGAN/out/output_21.png',
 '/home/sakib/DragGAN/out/output_44.png',
 '/home/sakib/DragGAN/out/output_24.png',
 '/home/sakib/DragGAN/out/output_27.png',
 '/home/sakib/DragGAN/out/output_25.png',
 '/home/sakib/DragGAN/out/output_46.png',
 '/home/sakib/DragGAN/out/output_40.png',
 '/home/sakib/DragGAN/out/output_49.png',
 '/home/sakib/DragGAN/out/output_36.pn

___

In [26]:
import re
from typing import List

def parse_input(input_str: str) -> List[List[float]]:
    """
    Parses a string of comma separated numbers possibly surrounded by various types of brackets.
    Converts them into list of pairs (as list of floats).
    
    Parameters
    ----------
    input_str : str
        The string to parse. It should contain comma separated numbers, possibly surrounded by various types of brackets.

    Returns
    -------
    List[List[float]]
        A list of lists, where each sub-list contains a pair of floats.
    
    Notes
    -----
    The function ignores invalid characters (non-numbers and non-commas) and handles numbers without a leading digit.

    Examples
    --------
    Normal cases:
    >>> parse_input("[(1,2), (3,4)]")
    [[1.0, 2.0], [3.0, 4.0]]

    >>> parse_input("[[1,2], [3,4]]")
    [[1.0, 2.0], [3.0, 4.0]]

    Multiple nested brackets:
    >>> parse_input("[[[1, 2], [3, 4]], [[5, 6], [7, 8]]]")
    [[1, 2], [3, 4], [5, 6], [7, 8]]

    Negative numbers:
    >>> parse_input("[-1, -2]")
    [[-1, -2]]

    Decimal points without leading digits:
    >>> parse_input("[.5, 1]")
    [[0.5, 1]]

    Whitespace variations:
    >>> parse_input("[1,     2]")
    [[1, 2]]

    >>> parse_input("[(1,  2),  (3,4)]")
    [[1, 2], [3, 4]]

    Invalid characters:
    >>> parse_input("a,b")
    [[0.0, 0.0]]
    """
    # sanitize the input, removing all brackets and only leaving numbers, commas, and minus signs
    sanitized = re.sub(r"[^\d.,-]", "", input_str)

    # split the sanitized input string by commas
    numbers_str = sanitized.split(',')
    
    # convert strings to int or float as necessary
    # Exclude empty strings and convert to float even for whole numbers for consistency
    numbers = [float(x) if x else 0.0 for x in numbers_str]  

    # group them into pairs and convert them into the format that you need (list of lists)
    # This ensures we always have pairs of numbers even if an odd number of numbers is provided
    return [numbers[i:i+2] for i in range(0, len(numbers), 2) if len(numbers[i:i+2]) == 2]

# Testing:
assert parse_input("[1,2]") == [[1, 2]]
assert parse_input("[[1,2]]") == [[1, 2]]
assert parse_input("[[1,2.0]]") == [[1, 2.0]]
assert parse_input("[1,2], [3,4]") == [[1, 2], [3, 4]]
assert parse_input("[1,    2], [3,  4]") == [[1, 2], [3, 4]]
assert parse_input("[[1,2], [3,4]]") == [[1, 2], [3, 4]]
assert parse_input("[(1,2), (3,4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1,  2), (3,4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1,2), [3,4]]") == [[1, 2], [3, 4]]
assert parse_input("[(1,2), (3,4))") == [[1, 2], [3, 4]]
assert parse_input("((1,2), (3,4))") == [[1, 2], [3, 4]]
assert parse_input("[(1,2), (3,4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1.2,2), (3,4)]") == [[1.2, 2], [3, 4]]

# Test for multiple nested brackets
assert parse_input("[[[1, 2], [3, 4]], [[5, 6], [7, 8]]]") == [[1, 2], [3, 4], [5, 6], [7, 8]]
assert parse_input("[[[1, 2], [3, 4]], [[5, 6], [7, 8]]][]") == [[1, 2], [3, 4], [5, 6], [7, 8]]
assert parse_input("[[[1, 2], [3, 4]], [[5, 6], [7, 8]]]") == [[1, 2], [3, 4], [5, 6], [7, 8]]
assert parse_input("[[[1, 2], [3, 4]], [[5, 6][], [7, 8]][]]") == [[1, 2], [3, 4], [5, 6], [7, 8]]
assert parse_input("[[[1, 2][], [3, 4]], [[5, 6], [7, 8]]]") == [[1, 2], [3, 4], [5, 6], [7, 8]]

# Test for mixed bracket types
assert parse_input("[1, 2], (3, 4)") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2], [3, 4)]") == [[1, 2], [3, 4]]
assert parse_input("[[1, 2], (3, 4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2), [3, 4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2], (3, 4)]") == [[1, 2], [3, 4]]

# Test for negative numbers
assert parse_input("[-1, -2]") == [[-1, -2]]
assert parse_input("[-1, -2.0]") == [[-1, -2.0]]
assert parse_input("[-1.2, -2]") == [[-1.2, -2]]
assert parse_input("[-1.2, -2.0]") == [[-1.2, -2.0]]
assert parse_input("[(-1, -2), (-3, -4)]") == [[-1, -2], [-3, -4]]

# Test for decimal points without leading digits
assert parse_input("[.5, 1]") == [[0.5, 1]]
assert parse_input("[1, .2]") == [[1, 0.2]]
assert parse_input("[(.5, 1), (2, .3)]") == [[0.5, 1], [2, 0.3]]

# Test for whitespace variations
assert parse_input("[1,2]") == [[1, 2]]
assert parse_input("[1,     2]") == [[1, 2]]
assert parse_input("[1, 2],  [3, 4]") == [[1, 2], [3, 4]]
assert parse_input("[(1,  2),  (3,4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2),\t[3,4]]") == [[1, 2], [3, 4]]

# Test for empty input
assert parse_input("") == []

# Test for non digits
assert parse_input("a,b") == [[0.0, 0.0]]

# Test for single digits
assert parse_input("[0, 1]") == [[0, 1]]
assert parse_input("[5, 9]") == [[5, 9]]
assert parse_input("[(7, 3), (2, 8)]") == [[7, 3], [2, 8]]

# Test for larger numbers
assert parse_input("[100, 200]") == [[100, 200]]
assert parse_input("[(1000, 2000), (3000, 4000)]") == [[1000, 2000], [3000, 4000]]

# Test for leading and trailing spaces
assert parse_input(" [1, 2]") == [[1, 2]]
assert parse_input("[1, 2] ") == [[1, 2]]
assert parse_input(" [1, 2] ") == [[1, 2]]
assert parse_input("[ 1, 2]") == [[1, 2]]
assert parse_input("[1, 2 ]") == [[1, 2]]
assert parse_input(" [ 1, 2 ] ") == [[1, 2]]

# Test for additional whitespace within brackets
assert parse_input("[ 1 , 2]") == [[1, 2]]
assert parse_input("[1 , 2 ]") == [[1, 2]]
assert parse_input("[ 1 , 2 ]") == [[1, 2]]
assert parse_input("[( 1 , 2 ), ( 3 , 4 )]") == [[1, 2], [3, 4]]
assert parse_input("[(1 , 2) , (3 , 4)]") == [[1, 2], [3, 4]]
assert parse_input("[( 1 , 2) , ( 3 , 4 )]") == [[1, 2], [3, 4]]

# Test for missing brackets
assert parse_input("1, 2") == [[1,2]]
assert parse_input("1, 2, 3, 4") == [[1,2], [3,4]]
assert parse_input("(1, 2), (3, 4)") == [[1,2], [3,4]]
assert parse_input("(1, 2), (3, 4), (5, 6)") == [[1,2], [3,4], [5,6]]

# Test for invalid characters
assert parse_input("[1, 2], [3, 4]") == [[1, 2], [3, 4]]
assert parse_input("[1, 2], [3, 4], [5, 6]") == [[1, 2], [3, 4], [5, 6]]
assert parse_input("[1, 2], (3, 4)") == [[1, 2], [3, 4]]
assert parse_input("[1, 2], (3, 4), [5, 6]") == [[1, 2], [3, 4], [5, 6]]
assert parse_input("[1, 2], (3, 4), [5, 6), (7, 8]") == [[1, 2], [3, 4], [5, 6], [7, 8]]

# Test for mismatched bracket pairs
assert parse_input("[1, 2), (3, 4]") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2], (3, 4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2), (3, 4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2), (3, 4))") == [[1, 2], [3, 4]]
assert parse_input("((1, 2), (3, 4))") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2), (3, 4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1.2, 2), (3, 4)]") == [[1.2, 2], [3, 4]]

# Test for invalid input format
assert parse_input("[1, 2], [3, 4]") == [[1, 2], [3, 4]]
assert parse_input("[1, 2], [(3, 4)]") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2), [3, 4]]") == [[1, 2], [3, 4]]
assert parse_input("[(1, 2), [(3, 4)]]") == [[1, 2], [3, 4]]

# Test for mix of integers and floats
assert parse_input("[1, 2.0]") == [[1, 2.0]]
assert parse_input("[(1.2, 2), (3, 4.5)]") == [[1.2, 2], [3, 4.5]]

